In [1]:
from torchvision import utils
from basic_fcn import *
from dataloader import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import time

In [2]:
train_dataset = CityScapesDataset(csv_file='./Data/train.csv')
val_dataset = CityScapesDataset(csv_file='./Data/val.csv')
test_dataset = CityScapesDataset(csv_file='./Data/test.csv')

_batch_size = 32
_num_workers = 4
iou_classes = [11, 20, 24, 26, 33]

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=_batch_size,
                          num_workers=_num_workers,
                          shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=_batch_size,
                          num_workers=_num_workers,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=_batch_size,
                          num_workers=_num_workers,
                          shuffle=True)

In [3]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        #torch.nn.init.xavier_uniform(m.bias.data)
        m.bias.data.zero_()
        
epochs     = 100
criterion = nn.CrossEntropyLoss()# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
fcn_model = FCN(n_class=n_class)
# for name, params in fcn_model.named_parameters():
#     print(name, params.shape, params.requires_grad)
    
fcn_model.apply(init_weights)
#fcn_model = torch.load('best_model')

optimizer = optim.Adam(fcn_model.parameters(), lr=5e-3)

In [ ]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()
    
def train():
    for epoch in range(epochs):
        ts = time.time()
        for iter, (X, tar, Y) in enumerate(train_loader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = X.cuda()   # Move your inputs onto the gpu
                labels = Y.cuda() # Move your labels onto the gpu
            else:
                inputs, labels = X, Y# Unpack variables into inputs and labels
            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if iter % 100 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item(), time.time() - ts))
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        torch.save(fcn_model, 'best_model')

        val(epoch)
        fcn_model.train()


def val(epoch):
    fcn_model.eval()
    #Complete this function - Calculate loss, accuracy and IoU for every epoch
    # Make sure to include a softmax after the output from your model
    total_loss = 0
    acc_num = 0
    acc_den = 0
    iou_num = len(iou_classes)*[0] 
    iou_den = len(iou_classes)*[0] 
    
    for iter, (X_val, tar_val, Y_val) in enumerate(val_loader):
        if use_gpu:
            inputs_val = X_val.cuda()   # Move your inputs onto the gpu
            labels_val = Y_val.cuda() # Move your labels onto the gpu
        else:
            inputs_val, labels_val = X_val, Y_val# Unpack variables into inputs and labels

        outputs_val = fcn_model(inputs_val)
        
        _, preds = torch.max(outputs_val, 1)
        
        loss_val = criterion(outputs_val, labels_val).item()
        
        preds_cpu = preds.cpu()
        Y_cpu = Y_val.cpu()
        
        total_loss += loss_val
        
        nn, dd = pixel_acc(preds_cpu, Y_cpu)
        acc_num += nn
        acc_den += dd
        
        for iou_cc_idx in range(len(iou_classes)):
            i_nn, i_dd = iou(preds_cpu, Y_cpu, iou_classes[iou_cc_idx])
            iou_num[iou_cc_idx] += i_nn
            iou_den[iou_cc_idx] += i_dd
            
        del inputs_val
        del labels_val
        del loss_val
        
    print("Validation loss after epoch {}, , loss: {}".format(epoch, total_loss))
    print("Accuracy on val after epoch {}, accuracy: {}".format(epoch, 100*acc_num/acc_den))
    for iou_cc_idx in range(len(iou_classes)):
        print("IOU for class {} on val after epoch{}, iou: {}".format(iou_classes[iou_cc_idx], epoch, 100*iou_num[iou_cc_idx]/iou_den[iou_cc_idx]))

    del total_loss 
    
def test():
    #Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    for iter, (X_tst, tar_tst, Y_tst) in enumerate(test_loader):
        if use_gpu:
            inputs_tst = X_tst.cuda()   # Move your inputs onto the gpu
            labels_tst = Y_tst.cuda() # Move your labels onto the gpu
        else:
            inputs_tst, labels_tst = X_tst, Y_tst# Unpack variables into inputs and labels

        outputs_tst = fcn_model(inputs_tst)
        loss_tst = criterion(outputs_tst, labels_tst)
        print("Validation loss after epoch {}, , loss: {}".format(epoch, loss_tst.item()))
        #print("Accuracy on test after epoch {}, accuracy: {}".format(epoch, pixel_acc(outputs_tst, Y_tst)))
        #print("IOU on test after epoch{}".format(epoch, iou(outputs_tst, Y_tst)))
            
        
        #Add accuracy and IoU
              
    
if __name__ == "__main__":
    val(0)  # show the accuracy before training
    train()

Validation loss after epoch 0, , loss: 56.421873569488525
Accuracy on val after epoch 0, accuracy: 2.757010803405478
IOU for class 11 on val after epoch0, iou: 8.149617833289138
IOU for class 20 on val after epoch0, iou: 0.14109853789703658
IOU for class 24 on val after epoch0, iou: 0.46219024976633877
IOU for class 26 on val after epoch0, iou: 0.02638342568287213
IOU for class 33 on val after epoch0, iou: 0.059036033645123026
epoch0, iter0, loss: 3.526369571685791
Finish epoch 0, time elapsed 83.7071840763092
Validation loss after epoch 0, , loss: 29.171831369400024
Accuracy on val after epoch 0, accuracy: 47.76181360515636
IOU for class 11 on val after epoch0, iou: 15.650300772645304
IOU for class 20 on val after epoch0, iou: 0.0
IOU for class 24 on val after epoch0, iou: 0.0
IOU for class 26 on val after epoch0, iou: 0.0
IOU for class 33 on val after epoch0, iou: 0.0
epoch1, iter0, loss: 4.0700836181640625
Finish epoch 1, time elapsed 84.89261841773987
Validation loss after epoch 1,